# A1111 Model Installer — SD 1.5 (UI)

Installs **SD 1.5** assets for Automatic1111 using a simple UI:

- **Checkpoints** (paste URLs) → `/workspace/a1111/models/Stable-diffusion/`
- **VAE (optional)** → `/workspace/a1111/models/VAE/`
- **ControlNet (SD15)** → `/workspace/a1111/models/ControlNet/`
- **ADetailer YOLO models** → `/workspace/a1111/models/adetailer/`
- **Upscalers (ESRGAN)** → `/workspace/a1111/models/ESRGAN/`

**VAE note:** Many SD 1.5 checkpoints already include a VAE or expect the default; leave the VAE box **unchecked** unless you specifically want **MSE VAE**.

Re-running is safe: existing files are skipped; existing repos are updated.

> If a model is gated on Hugging Face, set `HF_TOKEN` in the RunPod template.

In [ ]:
# First time? Uncomment, run once, then Kernel → Restart
# !pip install -q ipywidgets tqdm requests huggingface_hub
print("Downloader UI ready (install dependencies once if widgets are missing).")

In [ ]:
from pathlib import Path
import os

# A1111 data root (matches WEBUI_ROOT in your image)
DATA_ROOT = Path(os.environ.get("WEBUI_ROOT", "/workspace/a1111"))
CKPT_DIR = DATA_ROOT / "models/Stable-diffusion"
VAE_DIR  = DATA_ROOT / "models/VAE"
CN_DIR   = DATA_ROOT / "models/ControlNet"
ADET_DIR = DATA_ROOT / "models/adetailer"
ESRGAN_DIR = DATA_ROOT / "models/ESRGAN"

for d in (CKPT_DIR, VAE_DIR, CN_DIR, ADET_DIR, ESRGAN_DIR):
    d.mkdir(parents=True, exist_ok=True)

print("Data root:", DATA_ROOT)
print("↳ Checkpoints:", CKPT_DIR)
print("↳ VAE:", VAE_DIR)
print("↳ ControlNet (SD15):", CN_DIR)
print("↳ ADetailer models:", ADET_DIR)
print("↳ Upscalers (ESRGAN):", ESRGAN_DIR)

## Helpers (with progress bars)
Supports **Hugging Face** (repo + filename) and **direct URLs**.

In [ ]:
import os, shutil, requests, shlex, subprocess, time
from tqdm import tqdm
from huggingface_hub import hf_hub_download
from pathlib import Path

def human(n: int) -> str:
    for unit in ["B","KB","MB","GB"]:
        if n < 1024: return f"{n:.1f} {unit}"
        n /= 1024
    return f"{n:.1f} TB"

",
def download_direct(url: str, dest: Path):
    dest.parent.mkdir(parents=True, exist_ok=True)
    if dest.exists():
        print("✔ Exists:", dest); return dest
    r = requests.get(url, stream=True)
    r.raise_for_status()
    total = int(r.headers.get('Content-Length', 0))
    tmp = dest.with_suffix(dest.suffix + ".part")
    with open(tmp, 'wb') as f, tqdm(total=total, unit='B', unit_scale=True, desc=dest.name, ncols=80) as bar:
        for chunk in r.iter_content(chunk_size=1024*1024):
            if chunk:
                f.write(chunk); bar.update(len(chunk))
    tmp.replace(dest)
    print("✔ Saved:", dest, f"({human(dest.stat().st_size)})")
    return dest

def download_hf(repo_id: str, filename: str, dest_dir: Path, dest_name: str|None=None, token: str|None=None):
    dest = dest_dir / (dest_name or filename)
    dest.parent.mkdir(parents=True, exist_ok=True)
    if dest.exists():
        print("✔ Exists:", dest); return dest
    print(f"→ HF: {repo_id}/{filename}")
    path = hf_hub_download(
        repo_id=repo_id,
        filename=filename,
        token=token or os.environ.get("HF_TOKEN"),
        local_dir=dest_dir,
        local_dir_use_symlinks=False,
        resume_download=True,
    )
    if Path(path) != dest:
        shutil.copy2(path, dest)
    print("✔ HF downloaded:", dest, f"({human(dest.stat().st_size)})")
    return dest

print("Helpers loaded.")

## Pick what to install
- Paste SD 1.5 **checkpoint URLs** below (Civitai, HF, etc.). One per line.
- **VAE** is optional (unchecked).
- **ControlNet SD15**: Canny / SoftEdge / OpenPose pre-checked.
- **ADetailer**: Face YOLO pre-checked, Hand optional (you can edit URLs).
- **Upscalers**: 4x-UltraSharp pre-checked (editable URL).

In [ ]:
import ipywidgets as W
from IPython.display import display, Markdown

# --- VAE (optional) ---
vae_checkbox = W.Checkbox(
    description="Install SD1.5 MSE VAE (stabilityai/sd-vae-ft-mse)",
    value=False
)

# --- ControlNet SD15 ---
cn_options = [
    ("Canny",     "lllyasviel/control_v11p_sd15_canny"),
    ("Depth",     "lllyasviel/control_v11p_sd15_depth"),
    ("SoftEdge",  "lllyasviel/control_v11p_sd15_softedge"),
    ("LineArt",   "lllyasviel/control_v11p_sd15_lineart"),
    ("NormalBae", "lllyasviel/control_v11p_sd15_normalbae"),
    ("OpenPose",  "lllyasviel/control_v11p_sd15_openpose"),
]
prechecked = {"Canny","SoftEdge","OpenPose"}
cn_checkboxes = [W.Checkbox(description=label, value=(label in prechecked)) for label, _ in cn_options]

# --- ADetailer models (editable URLs) ---
adet_face_cb = W.Checkbox(description="ADetailer Face model", value=True)
adet_face_url = W.Text(
    value="https://huggingface.co/Bingsu/adetailer/resolve/main/models/face_yolov8n.pt",
    layout=W.Layout(width="100%")
)
adet_hand_cb = W.Checkbox(description="ADetailer Hand model", value=False)
adet_hand_url = W.Text(
    value="https://huggingface.co/Bingsu/adetailer/resolve/main/models/hand_yolov8n.pt",
    layout=W.Layout(width="100%")
)

# --- Upscalers (ESRGAN) ---
up_ultrasharp_cb = W.Checkbox(description="4x-UltraSharp (ESRGAN)", value=True)
up_ultrasharp_url = W.Text(
    value="https://huggingface.co/Bing-su/4x-UltraSharp/resolve/main/4x-UltraSharp.pth",
    layout=W.Layout(width="100%")
)

# --- User-provided checkpoint URLs ---
ckpt_text = W.Textarea(
    value="",
    placeholder="Paste one or more SD1.5 .safetensors URLs here (one per line).",
    layout=W.Layout(width="100%", height="140px")
)

btn = W.Button(description="Download Selected", button_style="success", icon="download")
out = W.Output()

def on_click(_):
    out.clear_output()
    with out:
        errors = []

        # 1) VAE (optional)
        if vae_checkbox.value:
            try:
                download_hf(
                    repo_id="stabilityai/sd-vae-ft-mse",
                    filename="vae-ft-mse-840000-ema-pruned.safetensors",
                    dest_dir=VAE_DIR,
                    dest_name="vae-ft-mse-840000-ema-pruned.safetensors",
                )
            except Exception as e:
                errors.append(("VAE", str(e)))

        # 2) ControlNet SD15
        for cb, (label, repo) in zip(cn_checkboxes, cn_options):
            if not cb.value: continue
            try:
                download_hf(
                    repo_id=repo,
                    filename="diffusion_pytorch_model.safetensors",
                    dest_dir=CN_DIR,
                    dest_name=f"control_v11p_sd15_{label.lower()}.safetensors",
                )
            except Exception as e:
                errors.append((f"ControlNet {label}", str(e)))

        # 3) ADetailer YOLO models (direct URLs, editable)
        try:
            if adet_face_cb.value and adet_face_url.value.strip():
                download_direct(adet_face_url.value.strip(), ADET_DIR / "face_yolov8n.pt")
        except Exception as e:
            errors.append(("ADetailer Face", str(e)))
        try:
            if adet_hand_cb.value and adet_hand_url.value.strip():
                download_direct(adet_hand_url.value.strip(), ADET_DIR / "hand_yolov8n.pt")
        except Exception as e:
            errors.append(("ADetailer Hand", str(e)))

        # 4) Upscalers (ESRGAN)
        try:
            if up_ultrasharp_cb.value and up_ultrasharp_url.value.strip():
                download_direct(up_ultrasharp_url.value.strip(), ESRGAN_DIR / "4x-UltraSharp.pth")
        except Exception as e:
            errors.append(("4x-UltraSharp", str(e)))

        # 5) Checkpoints (direct URLs)
        urls = [u.strip() for u in ckpt_text.value.splitlines() if u.strip()]
        for url in urls:
            try:
                name = url.split("/")[-1].split("?")[0]
                if not (name.endswith(".safetensors") or name.endswith(".ckpt")):
                    name += ".safetensors"
                download_direct(url, CKPT_DIR / name)
            except Exception as e:
                errors.append((url, str(e)))

        # Summary
        print("\n=== Summary ===")
        any_files = False
        for p in sorted(list(CKPT_DIR.glob('*')) + list(VAE_DIR.glob('*')) + list(CN_DIR.glob('*')) + list(ADET_DIR.glob('*')) + list(ESRGAN_DIR.glob('*'))):
            try:
                print(f"✔ {p.name:45s} {os.path.getsize(p):,} B"); any_files = True
            except Exception:
                pass
        if not any_files:
            print("(No new files)")
        if errors:
            print("\nErrors:")
            for name, msg in errors:
                print("✖", name, "→", msg)

btn.on_click(on_click)

display(Markdown("### SD 1.5 Downloads"))
display(Markdown("**Checkpoints (paste URLs):**"))
display(ckpt_text)

display(Markdown("**VAE (optional):**"))
display(vae_checkbox)

display(Markdown("**ControlNet (SD15):**"))
for cb in cn_checkboxes:
    display(cb)

display(Markdown("**ADetailer models (editable URLs):**"))
display(adet_face_cb, adet_face_url)
display(adet_hand_cb, adet_hand_url)

display(Markdown("**Upscalers (ESRGAN):**"))
display(up_ultrasharp_cb, up_ultrasharp_url)

display(btn, out)

## (Optional) Restart the WebUI
Use this after new models are added so A1111 refreshes file lists.

In [ ]:
import subprocess, shlex, time
try:
    cmd = "supervisorctl -c /etc/supervisord.conf restart a1111"
    out = subprocess.run(shlex.split(cmd), capture_output=True, text=True, timeout=20)
    if out.returncode == 0:
        print("✔ Restarted via supervisord. Wait ~10–15s, then refresh the WebUI (7860).\n")
        print(out.stdout.strip())
    else:
        print("Supervisorctl failed; trying process restart…")
        subprocess.run(["pkill", "-f", "launch.py"], check=False)
        time.sleep(5)
        print("✔ WebUI killed; supervisord will auto-restart. Refresh in ~10–15s.")
except Exception as e:
    print("Restart attempt hit an error:", e)
    print("Manual command:")
    print("supervisorctl -c /etc/supervisord.conf restart a1111")